In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
actions = [
    'come',
    'away',
    'spin'
]

data = np.concatenate([
    np.load('dataset/seq_고민_1680106580.npy'),
    np.load('dataset/seq_away_1679649683.npy'),
    np.load('dataset/seq_spin_1679649683.npy')
], axis=0)

data.shape

(8095, 30, 100)

In [3]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(8095, 30, 99)
(8095,)


In [4]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

(8095, 3)

In [5]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(7285, 30, 99) (7285, 3)
(810, 30, 99) (810, 3)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                41984     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 44,163
Trainable params: 44,163
Non-trainable params: 0
_________________________________________________________________


2023-04-01 23:40:00.047503: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 23:40:00.049018: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

Train on 7285 samples, validate on 810 samples
Epoch 1/200
7264/7285 [============================>.] - ETA: 0s - loss: 7.3915 - acc: 0.5023
Epoch 00001: val_acc improved from -inf to 0.56049, saving model to models/model.h5
7285/7285 [==============================] - 15s 2ms/sample - loss: 7.3771 - acc: 0.5024 - val_loss: 2.8352 - val_acc: 0.5605
Epoch 2/200
7264/7285 [============================>.] - ETA: 0s - loss: 88.6273 - acc: 0.4931
Epoch 00002: val_acc did not improve from 0.56049
7285/7285 [==============================] - 13s 2ms/sample - loss: 88.4977 - acc: 0.4929 - val_loss: 29.5429 - val_acc: 0.4815
Epoch 3/200
7264/7285 [============================>.] - ETA: 0s - loss: 142.6725 - acc: 0.4668
Epoch 00003: val_acc did not improve from 0.56049
7285/7285 [==============================] - 13s 2ms/sample - loss: 142.4054 - acc: 0.4660 - val_loss: 136.3976 - val_acc: 0.2519
Epoch 4/200
7264/7285 [============================>.] - ETA: 0s - loss: 36.6161 - acc: 0.4818
Epoch

In [8]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [9]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))

array([[[172, 151],
        [152, 335]],

       [[570,  75],
        [ 71,  94]],

       [[575,  77],
        [ 80,  78]]])